In [6]:
import pandas as pd
import geopandas as gpd
import json
from tabulate import tabulate

In [ ]:
#pd.set_option('display.max_colwidth', None)
#pd.reset_option('display.max_colwidth')

In [21]:
segments_file = '/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/2021_12_22_sdegdw_f66_d_StrassenSegmente.geojson'
segments_clipped_file = '/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/strassensegmente_clip_kreuzung.json'
area_file_merged = '/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/leipzig_verkehrsflaechen_breite_gdi_merged_20230504.json'
ot_file = '/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/leipzig_ortsteile.json'

## merge breite into verkehrsflaechen json #TODO DELETE? haben wirn icht selber gemacht

In [ ]:
# # --- specify file paths -----#
# area_file = '/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/leipzig_verkehrsflaechen.json'
# area_file_with_breite = '/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/leipzig_verkehrsflaechen_breite.json'
# # --------------#   
# # Load the JSON file into a Python object
# with open(area_file_with_breite, 'r') as f:
#     breite_data = json.load(f)

# #Extract the "features" array from the JSON data
# breite_features = breite_data['features']

# #Normalize the "properties" object for each feature and create a Pandas dataframe
# breite_df = pd.concat([pd.json_normalize(item) for item in breite_features], axis=0)
# print(breite_df.columns)

# with open(area_file, 'rb') as f1:
#     area_data = json.load(f1)
# area_features = area_data['features']

# #Normalize the "properties" object for each feature and create a Pandas dataframe
# area_df = pd.concat([pd.json_normalize(item) for item in area_features], axis=0)
# area_df
# area_df['properties.median_bre'] = np.nan
# print(area_df.columns)

In [40]:
# feature_list = []
# for index, row in area_df.iterrows():
#     obj_id = row['properties.objectid']

#     try:
#         median_bre = breite_df.loc[breite_df['properties.objectid']==obj_id, 'properties.median_bre'].values[0]
#     except IndexError:
#         median_bre = -1
  
#     #area_df.loc[area_df['properties.objectid'] == obj_id,'properties.median_bre'] = median_bre
#     area_df.at[index, 'properties.median_bre'] = median_bre
#     properties = dict()
#     for col in area_df.columns:
#         if "properties" in col:
#             key = col.split(".")[1]
#             properties[key] = row[col]

#     feature = {"type" : "Feature",
#                "geometry" : {
#                    "type" : row['geometry.type'],
# 				    "coordinates" : row['geometry.coordinates']
#                     },
#                "properties" :  properties
#                }
#     feature_list.append(feature)



# json_res = {
# 	"type" : "FeatureCollection",
# 	"name" : "leipzig_verkehrsflaechen",
# 	"features" : feature_list
# }
# # Write the normalized DataFrame back to a JSON file
# with open("test.json", 'w', encoding="utf-8") as f2:
#     json.dump(json_res, f2, ensure_ascii=False)


# Read data and preprocess

### segments_clipped

In [45]:
# extract data from strassen_segmente_testgebiet.json - file the city sent us
def extract_segment_data(segments_json):
    street_items = []
    for feature in segments_json['features']:
        geom_type = feature['geometry']['type']
        if geom_type == "GeometryCollection":
            coordinates = feature['geometry']['geometries'][0]['coordinates']
            coordinates = [(item[1], item[0]) for item in coordinates[0]]
            coordinates = [convert_coords("EPSG:4326", "EPSG:25833", coords[0], coords[1]) for coords in coordinates]
            print("missing coordinates") if len(feature['geometry']['geometries']) > 1 else None
        else:
            coordinates = feature['geometry']['coordinates']
            coordinates = [(item[1], item[0]) for item in coordinates[0]]
            coordinates = [convert_coords("EPSG:4326", "EPSG:25833", coords[0], coords[1]) for coords in coordinates]
        str = feature['properties']['str']
        segm_gid = feature['properties']['segm_gid']
        objectid = feature['properties']['objectid']
        von_str = feature['properties']['von_str']
        bis_str = feature['properties']['bis_str']
        str_nr = feature['properties']['str_nr']
        kat_sstrgs = feature['properties']['kat_sstrgs']
        sstrgsname = feature['properties']['sstrgsname']
        laenge_m = feature['properties']['laenge_m']
        street_items.append({'str_nr': str_nr, 'str': str,'objectid': objectid, 'segm_gid': segm_gid, 'kat_sstrgs': kat_sstrgs, 'sstrgsname': sstrgsname, 'von_str': von_str, 'bis_str': bis_str, 'laenge_m': laenge_m, 'geom_type': geom_type, 'coordinates': coordinates})
    
    street_df = pd.DataFrame.from_records(street_items)
    return street_df


# write segments data from pandas df back to json
def write_segments_to_Json(segments_df):
    feature_list = []
    for index, row in segments_df.iterrows():
        
        properties = {'str_nr': row['str_nr'], 'str': row['str'],'objectid': row['objectid'], 'segm_gid': row['segm_gid'], 'kat_sstrgs': row['kat_sstrgs'], 'sstrgsname': row['sstrgsname'], 'von_str': row['von_str'], 'bis_str': row['bis_str'], 'laenge_m': row['laenge_m']}
     
        feature = {"type" : "Feature",
                "geometry" : {
                    "type" : row['geom_type'],
                     "coordinates" : row['coordinates']
                        },
                "properties" :  properties
                }
        feature_list.append(feature)
    return feature_list

In [13]:
# using geopandas
segments_clipped_gdf = gpd.read_file(segments_clipped_file, driver="GeoJSON")
print("Coordinate Reference System (CRS):", segments_clipped_gdf.crs)
print(segments_clipped_gdf.columns)
segments_clipped_gdf.head(3)

Coordinate Reference System (CRS): EPSG:4326
Index(['objectid', 'gid', 'segm_gid', 'str', 'str_nr', 'abs_nr17', 'abs_nr',
       'von_str', 'von_str_nr', 'bis_str', 'bis_str_nr', 'stadtbez',
       'klass_netz', 'kat_sstrgs', 'sstrgsname', 'kat_erschl', 'erschlname',
       'baulast', 'laenge_m', 'ortsdurchf', 'ortsd_text', 'hotlink',
       'fme_tstamp', 'st_length_', 'TDrei', 'OBJECTID_1', '_clipped',
       'geometry'],
      dtype='object')


,objectid,gid,segm_gid,str,str_nr,abs_nr17,abs_nr,von_str,von_str_nr,bis_str,...,laenge_m,ortsdurchf,ortsd_text,hotlink,fme_tstamp,st_length_,TDrei,OBJECTID_1,_clipped,geometry
0,1,442,442,Kändlerstraße,06242,06242062420622400,4,Kändlerstraße,06242,Krakauer Straße,...,138,,,,20211219000000+00:00,137.743623,,1,yes,"MULTILINESTRING ((12.26829 51.31232, 12.26991 ..."
1,2,608,608,Schrammsteinstraße,06246,06246062380624400,2,Basteistraße,06238,Liliensteinstraße,...,115,,,,20211219000000+00:00,114.583242,,2,yes,"MULTILINESTRING ((12.26858 51.30753, 12.26860 ..."
2,3,672,672,Bärlauchweg,04294,04294000000429400,1,Stadtgrenze,00000,Bärlauchweg,...,28,,,,20211219000000+00:00,27.891422,,3,no,"MULTILINESTRING ((12.36449 51.28812, 12.36466 ..."


In [7]:
print(segments_clipped_gdf['sstrgsname'].value_counts())

Ortsstraße                             9811
Beschränkt öffentl. Weg und Platz      1512
Bundesstraße                            313
Kreisstraße                             239
Staatsstraße                            185
Feld- und Waldweg                       157
Gemeindeverbindungsstraße                66
Eigentümerweg                            59
Sonstige (z.B. noch nicht gewidmet)      18
Autobahn                                  2
Name: sstrgsname, dtype: int64


#### Process df to only contain the needed data- Choose needed columns, convert coords if needed, choose specific values in df

In [14]:
columns = ['objectid', 'segm_gid', 'str', 'str_nr', 'von_str', 'bis_str', 'laenge_m', 'kat_sstrgs', 'sstrgsname', 'geometry']

# convert coords
segments_clipped_gdf = segments_clipped_gdf.to_crs(epsg=25833)
segments_clipped_gdf = segments_clipped_gdf[columns]


# remove parkplatz
initial_length = len(segments_clipped_gdf)
segments_clipped_gdf = segments_clipped_gdf.loc[(segments_clipped_gdf['sstrgsname'] == "Ortsstraße") | (segments_clipped_gdf['sstrgsname'] == "Bundesstraße") | (segments_clipped_gdf['sstrgsname'] == "Kreisstraße")  | (segments_clipped_gdf['sstrgsname'] == "Staatsstraße")]
print(f"Removed {initial_length - len(segments_clipped_gdf)} items")
new_length = len(segments_clipped_gdf)

segments_clipped_gdf = segments_clipped_gdf.loc[~segments_clipped_gdf['str'].str.contains("Parkplatz")]
print(f"Removed {new_length - len(segments_clipped_gdf)} items")

segments_clipped_gdf

#change geometry type #TODO this doesnt apply yet for GPD
# segments_clipped_gdf['geom_type'] = segments_clipped_gdf['geom_type'].replace("MultiLineString", "LineString")
# segments_clipped_gdf['geom_type'] = segments_clipped_gdf['geom_type'].replace("GeometryCollection", "LineString")

Removed 1814 items
Removed 0 items


,objectid,segm_gid,str,str_nr,von_str,bis_str,laenge_m,kat_sstrgs,sstrgsname,geometry
0,1,442,Kändlerstraße,06242,Kändlerstraße,Krakauer Straße,138,O,Ortsstraße,"MULTILINESTRING ((309622.183 5688100.624, 3097..."
1,2,608,Schrammsteinstraße,06246,Basteistraße,Liliensteinstraße,115,O,Ortsstraße,"MULTILINESTRING ((309623.168 5687567.487, 3096..."
2,3,672,Bärlauchweg,04294,Stadtgrenze,Bärlauchweg,28,O,Ortsstraße,"MULTILINESTRING ((316228.864 5685164.361, 3162..."
3,4,695,Am Wolfswinkel,05171,Bärlauchweg,Stadtgrenze,18,O,Ortsstraße,"MULTILINESTRING ((316207.054 5685351.221, 3162..."
4,5,736,Am Eichenbogen,04142,Stadtgrenze,Stadtgrenze,204,O,Ortsstraße,"MULTILINESTRING ((316256.367 5685133.583, 3162..."
...,...,...,...,...,...,...,...,...,...,...
12353,12808,32361,Kresseweg,09354,Kresseweg,Basilikumweg,37,O,Ortsstraße,"MULTILINESTRING ((317319.679 5697934.848, 3173..."
12354,12809,32362,Kresseweg,09354,Kresseweg,Salbeiweg,63,O,Ortsstraße,"MULTILINESTRING ((317280.941 5698005.863, 3172..."
12355,12810,32363,Kresseweg,09354,Kresseweg,Basilikumweg,77,O,Ortsstraße,"MULTILINESTRING ((317299.341 5698010.777, 3173..."
12357,12812,32365,Basilikumweg,09316,Thymianweg,Kresseweg,32,O,Ortsstraße,"MULTILINESTRING ((317369.099 5698047.377, 3173..."


#### save file

In [9]:
out_file = '../datasets/complete/2021_12_22_sdegdw_f66_d_StrassenSegmente_CLEANED_CLIPPED2.geojson'
segments_clipped_gdf.to_file(out_file, driver="GeoJSON")

### segments (old file)

In [11]:
segments_gdf = gpd.read_file(segments_file, driver="GeoJSON")
print("Coordinate Reference System (CRS):", segments_gdf.crs)
print(segments_gdf.columns)

Coordinate Reference System (CRS): EPSG:25833
Index(['FID', 'objectid', 'gid', 'segm_gid', 'str', 'str_nr', 'abs_nr17',
       'abs_nr', 'von_str', 'von_str_nr', 'bis_str', 'bis_str_nr', 'stadtbez',
       'klass_netz', 'kat_sstrgs', 'sstrgsname', 'kat_erschl', 'erschlname',
       'baulast', 'laenge_m', 'ortsdurchf', 'ortsd_text', 'hotlink',
       'fme_tstamp', 'st_length_', 'TDrei', 'geometry'],
      dtype='object')


In [12]:
columns = ['objectid', 'segm_gid', 'str', 'str_nr', 'von_str', 'bis_str', 'laenge_m', 'kat_sstrgs', 'sstrgsname', 'geometry']

# convert coords
#segments_clipped_gdf = segments_clipped_gdf.to_crs(epsg=25833)
segments_gdf = segments_gdf[columns]

# remove parkplatz
segments_gdf = segments_gdf.loc[(segments_gdf['sstrgsname'] == "Ortsstraße") | (segments_gdf['sstrgsname'] == "Bundesstraße") | (segments_gdf['sstrgsname'] == "Kreisstraße")  | (segments_gdf['sstrgsname'] == "Staatsstraße")]
segments_gdf

,objectid,segm_gid,str,str_nr,von_str,bis_str,laenge_m,kat_sstrgs,sstrgsname,geometry
0,1,442,Kändlerstraße,06242,Kändlerstraße,Krakauer Straße,138,O,Ortsstraße,"LINESTRING (309616.483 5688099.341, 309750.863..."
1,2,608,Schrammsteinstraße,06246,Basteistraße,Liliensteinstraße,115,O,Ortsstraße,"LINESTRING (309605.269 5687557.655, 309624.163..."
2,3,672,Bärlauchweg,04294,Stadtgrenze,Bärlauchweg,28,O,Ortsstraße,"LINESTRING (316228.864 5685164.361, 316241.556..."
3,4,695,Am Wolfswinkel,05171,Bärlauchweg,Stadtgrenze,18,O,Ortsstraße,"LINESTRING (316207.054 5685351.221, 316222.024..."
4,5,736,Am Eichenbogen,04142,Stadtgrenze,Stadtgrenze,204,O,Ortsstraße,"LINESTRING (316256.367 5685133.583, 316274.622..."
...,...,...,...,...,...,...,...,...,...,...
12807,12808,32361,Kresseweg,09354,Kresseweg,Basilikumweg,37,O,Ortsstraße,"LINESTRING (317310.779 5697933.351, 317317.790..."
12808,12809,32362,Kresseweg,09354,Kresseweg,Salbeiweg,63,O,Ortsstraße,"LINESTRING (317290.141 5698008.360, 317229.221..."
12809,12810,32363,Kresseweg,09354,Kresseweg,Basilikumweg,77,O,Ortsstraße,"LINESTRING (317290.141 5698008.360, 317357.147..."
12811,12812,32365,Basilikumweg,09316,Thymianweg,Kresseweg,32,O,Ortsstraße,"LINESTRING (317370.800 5698056.877, 317365.217..."


### areas

In [6]:
# manually read json data and wrte back to Json
def extract_area_data(segments_json):
    street_items = []
    for feature in segments_json['features']:
        geom_type = feature['geometry']['type']
        coordinates = feature['geometry']['coordinates'][0]
        #print(coords)
        if geom_type == "MultiPolygon":
            for coord_list in coordinates:
                coord_list = [(item[1], item[0]) for item in coord_list]
        else:
            coordinates = [(item[1], item[0]) for item in coordinates]
        str = feature['properties']['str']
        objectid = feature['properties']['objectid']
        stsa_text = feature['properties']['stsa_text']
        segm_gid = feature['properties']['segm_gid']
        laenge = feature['properties']['laenge']
        median_breite = feature['properties']['median_breite']

        street_items.append({'str': str,'objectid': objectid, 'stsa_text': stsa_text, 'segm_gid': segm_gid, 'laenge': laenge, 'median_breite': median_breite, 'geom_type': geom_type, 'coordinates': coordinates})
    
    street_df = pd.DataFrame.from_records(street_items)
    return street_df


def write_area_to_Json(area_df):
    feature_list = []
    for index, row in area_df.iterrows():
        
        properties = {'str': row['str'],'objectid': row['objectid'], 'stsa_text': row['stsa_text'], 'segm_gid': row['segm_gid'],'median_breite': row['median_breite'], 'laenge': row['laenge']}
     
        feature = {"type" : "Feature",
                "geometry" : {
                    "type" : row['geom_type'],
                     "coordinates" : row['coordinates']
                        },
                "properties" :  properties
                }
        feature_list.append(feature)
    return feature_list

In [18]:
area_gdf = gpd.read_file(area_file_merged, driver="GeoJSON")
print("Coordinate system is: ",area_gdf.crs)
print(area_gdf['stsa_text'].value_counts())
area_gdf.head()

Coordinate system is:  EPSG:4326
Gehweg                                                                        17119
Grünfläche                                                                    15205
Fahrbahn (dazu gehören Parkbuchten, Stellflächen, Radstreifen, Busbuchten)    13015
Seitenstreifen, Randstreifen, Bankett                                          4240
Seitentrennstreifen                                                            1793
Straßengraben, Straßenmulde                                                    1371
Querungshilfe                                                                  1212
Radweg                                                                         1145
Straßenbündiger Bahnkörper                                                     1128
Böschung                                                                       1112
Geh- und Radweg                                                                 686
Verkehrsteiler                             

,objectid,segm,laenge,breite,str,stsa_key,sdbf_gid,gid,flaeche,lage,...,stgrp_bez,hotlink,fme_tstamp,st_area(shape),st_length(shape),objectid_verkehrsflaeche,mean_breite,median_breite,anzahl_punkte_berechnung,geometry
0,1796085,02047020860208700.10/1,156,1.46,Delitzscher Straße,24,None,None,284.23,R,...,Geh+Radweg,None,20230430065010,284.226005,272.086204,1796085,2.08,2.14,148,"POLYGON ((12.38586 51.37263, 12.38585 51.37263..."
1,1796086,02047020860208700.10/1,156,1.46,Delitzscher Straße,24,None,None,284.02,L,...,Geh+Radweg,None,20230430065010,284.005451,276.843633,1796086,2.04,1.97,87,"POLYGON ((12.38554 51.37268, 12.38551 51.37268..."
2,1796087,02047020860208700.10/1,156,2.43,Delitzscher Straße,21,None,None,351.55,R,...,Geh+Radweg,None,20230430065010,351.544150,293.816643,1796087,2.38,2.36,98,"POLYGON ((12.38589 51.37265, 12.38588 51.37264..."
3,1796088,02047020860208700.10/1,156,2.43,Delitzscher Straße,21,None,None,405.09,L,...,Geh+Radweg,None,20230430065010,405.091807,317.995039,1796088,2.53,2.56,60,"POLYGON ((12.38552 51.37255, 12.38551 51.37268..."
4,1796089,02047020870803700.10/1,117,5.94,Delitzscher Straße,34,None,None,652.69,M,...,entfällt in der Bewertung,None,20230430065010,652.686513,227.990146,1796089,5.49,5.97,88,"POLYGON ((12.38562 51.37366, 12.38561 51.37367..."


In [20]:
# remove all others except Fahrbahn
initial_length = len(area_gdf)
area_gdf = area_gdf.loc[area_gdf['stsa_text'].str.contains("Fahrbahn")]
print(f"Removed {initial_length - len(area_gdf)} items.")

#convert coords
area_gdf = area_gdf.to_crs(epsg=25833)
area_gdf.head(3)

Removed 0 items.


,objectid,segm,laenge,breite,str,stsa_key,sdbf_gid,gid,flaeche,lage,...,stgrp_bez,hotlink,fme_tstamp,st_area(shape),st_length(shape),objectid_verkehrsflaeche,mean_breite,median_breite,anzahl_punkte_berechnung,geometry
5,1796090,02047020870803700.10/1,117,3.47,Delitzscher Straße,10,None,None,737.32,R,...,Fahrbahn,None,20230430065010,737.304393,274.885587,1796090,5.40,5.95,97,"POLYGON ((318041.680 5694631.273, 318041.787 5..."
9,1796094,02047080370208900.10/1,78,3.42,Delitzscher Straße,10,None,None,396.08,R,...,Fahrbahn,None,20230430065010,396.070962,183.949100,1796094,4.22,3.48,79,"POLYGON ((318040.652 5694709.132, 318040.724 5..."
15,1796099,02047020890208800.10/1,190,3.54,Delitzscher Straße,10,None,None,1328.46,R,...,Fahrbahn,None,20230430065010,1328.435608,416.375444,1796099,6.36,7.29,136,"POLYGON ((318037.192 5694899.342, 318037.158 5..."


In [ ]:
# write new file
out_filepath = ""
area_gdf.to_file(out_filepath, driver="GeoJSON")

## convert coordinates of ortsteile file from 4326 to 25833 for postgis

In [23]:
ot_gdf = gpd.read_file(ot_file, driver="GeoJSON")
print("Coordinate system is: ",ot_gdf.crs)
print(ot_gdf.columns)

#convert coords
ot_gdf = ot_gdf.to_crs(epsg=25833)
ot_gdf.head(3)

Coordinate system is:  EPSG:4326
Index(['objectid', 'ot_nr', 'ot_name', 'sbz_nr', 'sbz_name', 'geometry'], dtype='object')


,objectid,ot_nr,ot_name,sbz_nr,sbz_name,geometry
0,1,00,Zentrum,0,Mitte,"POLYGON ((317483.317 5691380.912, 317472.492 5..."
1,2,01,Zentrum-Ost,0,Mitte,"POLYGON ((318071.542 5690670.373, 318096.179 5..."
2,3,02,Zentrum-Südost,0,Mitte,"POLYGON ((318544.743 5690680.070, 318550.703 5..."


In [ ]:
# write new file
out_filepath = "/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/leipzig_ortsteile_CLEANED.json"
ot_gdf.to_file(out_filepath, driver="GeoJSON")

# old

In [ ]:
#Edit segment file with clipped Kreuzungen: keep only Ortsstraße, Bundestraße, Kreisstraße (to remove Parkplatz) and convert coords
with open(segments_clipped_file, 'rb') as f2:
    segments_clipped_json = json.load(f2)

#extract segments
segments_clipped_df = extract_segment_data(segments_clipped_json)

# remove parkplatz
segments_clipped_df = segments_clipped_df.loc[(segments_clipped_df['sstrgsname'] == "Ortsstraße") | (segments_clipped_df['sstrgsname'] == "Bundesstraße") | (segments_clipped_df['sstrgsname'] == "Kreisstraße")]
#segments_clipped_df['geom_type'] == "LineString"
segments_clipped_df['geom_type'] = segments_clipped_df['geom_type'].replace("MultiLineString", "LineString")
segments_clipped_df['geom_type'] = segments_clipped_df['geom_type'].replace("GeometryCollection", "LineString")
print(tabulate(segments_clipped_df.head(), headers='keys', tablefmt='psql'))

feature_list = write_segments_to_Json(segments_clipped_df)
json_res = {
	"type" : "FeatureCollection",
	"name" : "leipzig_strassensegmente_clipped",
	"features" : feature_list
}
# Write the normalized DataFrame back to a JSON file
with open("/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/2021_12_22_sdegdw_f66_d_StrassenSegmente_CLEANED_CLIPPED.json", 'w', encoding="utf-8") as seg_file:
    json.dump(json_res, seg_file, ensure_ascii=False)

In [ ]:
#edit area file: keep only fahrbahn
with open(area_file_merged, "rb") as f2:
    area_json = json.load(f2)
area_df = extract_area_data(area_json)
area_df= area_df.loc[area_df['stsa_text'].str.contains("Fahrbahn")]
print(area_df.head())

feature_list = write_area_to_Json(area_df)
json_area_new = {
	"type" : "FeatureCollection",
	"name" : "vta_verkehrsflaechen_breite_20230504",
    "features": feature_list }

# Write the normalized DataFrame back to a JSON file
with open("/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/leipzig_verkehrsflaechen_breite_gdi_merged_20230504_CLEANED.json", 'w', encoding="utf-8") as f2:
    json.dump(json_area_new, f2, ensure_ascii=False)